# Build IMDb actor dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv
from ast import literal_eval
from project_utils import *
from pandas.io.json import json_normalize
from functools import reduce

%reload_ext lab_black

In [2]:
credits_df = pd.read_csv("./data/tmdb_5000_credits.csv", sep=",", quotechar='"')
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
movies_df = pd.read_csv("./data/tmdb_5000_movies.csv", sep=",", quotechar='"').rename(
    {"id": "movie_id"}, axis=1
)
movies_df.head()

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
movies_df = pd.read_csv("./data/tmdb_5000_movies.csv", sep=",", quotechar='"').rename(
    {"id": "movie_id"}, axis=1
)
movies_df.head()

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


## Process movies DataFrame
Flatten and clean movies DataFrame

In [5]:
pr_movies_df = movies_df.copy()
# Transform json to list of dictionaries within cols
movies_df_json_cols = [
    "genres",
    "production_companies",
    "keywords",
    "production_countries",
    "spoken_languages",
]
cols_to_drop = ["homepage", "overview", "tagline", "original_title"]
key = "name"
movie_id_col = "movie_id"
pr_movies_df = col_json_to_dict(pr_movies_df, movies_df_json_cols)

# Transform list of dictionaries to sets containing a relevant value from dictionaries
for col in movies_df_json_cols:
    pr_movies_df = col_dict_to_set(pr_movies_df, col, key)
pr_movies_df = pr_movies_df.drop(columns=cols_to_drop)
pr_movies_df.head()

,budget,genres,movie_id,keywords,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,237000000,"{Fantasy, Action, Adventure, Science Fiction}",19995,"{3d, alien, cgi, space war, mind and soul, fut...",en,150.437577,"{Lightstorm Entertainment, Dune Entertainment,...","{United Kingdom, United States of America}",2009-12-10,2787965087,162.0,"{English, Español}",Released,Avatar,7.2,11800
1,300000000,"{Fantasy, Action, Adventure}",285,"{love of one's life, shipwreck, traitor, ship,...",en,139.082615,"{Walt Disney Pictures, Second Mate Productions...",{United States of America},2007-05-19,961000000,169.0,{English},Released,Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"{Action, Adventure, Crime}",206647,"{sequel, british secret service, secret agent,...",en,107.376788,"{Danjaq, Columbia Pictures, B24}","{United Kingdom, United States of America}",2015-10-26,880674609,148.0,"{Italiano, Español, English, Deutsch, Français}",Released,Spectre,6.3,4466
3,250000000,"{Drama, Action, Thriller, Crime}",49026,"{hostage drama, destruction, burglar, time bom...",en,112.312950,"{DC Entertainment, Legendary Pictures, Syncopy...",{United States of America},2012-07-16,1084939099,165.0,{English},Released,The Dark Knight Rises,7.6,9106
4,260000000,"{Action, Adventure, Science Fiction}",49529,"{3d, superhuman strength, alien race, space tr...",en,43.926995,{Walt Disney Pictures},{United States of America},2012-03-07,284139100,132.0,{English},Released,John Carter,6.1,2124


## Process credits DataFrame
Flatten and clean credits DataFrame

In [6]:
pr_credits_df = credits_df.copy()

# Parse json cols to list of dictionaries
credits_df_json_cols = ["cast", "crew"]
key = "name"
pr_credits_df = col_json_to_dict(pr_credits_df, credits_df_json_cols)
# Create the actors col
pr_credits_df = pr_credits_df.assign(actors=pr_credits_df[credits_df_json_cols[0]])
# Get sets of relevant values from the lists of dictionaries
for col in credits_df_json_cols:
    if col != "crew":
        pr_credits_df = col_dict_to_set(pr_credits_df, col, key)

In [7]:
# Filter crew dictionaries with Director as job
crew_col = "crew"
job_field = "job"
values = ["Director"]
pr_credits_df = col_filter_dict_with_vals(
    pr_credits_df, crew_col, job_field, values
).rename({crew_col: values[0]})
pr_credits_df = col_dict_to_set(pr_credits_df, crew_col, key)

In [8]:
# Select a specific number of actors as nodes
actors_col = "actors"
order_field = "order"
# Set the actor importances (the order key. The main actor is order 0) to consider for taking an actor as a node
values = [0]
pr_credits_df = col_filter_dict_with_vals(
    pr_credits_df, actors_col, order_field, values
).rename({crew_col: values[0]})

get_value = lambda dict_: dict_.get(key)

pr_credits_df = pr_credits_df.explode(actors_col)
pr_credits_df = pr_credits_df[pr_credits_df[actors_col].notna()]
actors_series = pr_credits_df[actors_col]
pr_credits_df = pr_credits_df.assign(
    **{actors_col: pr_credits_df[actors_col].apply(get_value)}
)

In [9]:
# Remove actors taken as nodes from cast column
actor_cast_col = "actor_cast"
pr_credits_df = pr_credits_df.reset_index(drop=True)
remove_from_cast = lambda row: row[credits_df_json_cols[0]] - set({row[actors_col]})
pr_credits_df = pr_credits_df.assign(
    **{credits_df_json_cols[0]: pr_credits_df.apply(remove_from_cast, axis=1)}
).drop(columns="title")
pr_credits_df.head()

,movie_id,cast,crew,actors
0,19995,"{Tamica Washington-Miller, Hanniyah Muhammad, ...",{James Cameron},Sam Worthington
1,285,"{Takayo Fischer, Ho-Kwan Tse, Mackenzie Crook,...",{Gore Verbinski},Johnny Depp
2,206647,"{Adriana Paz, Jesper Christensen, Wilhem Iben,...",{Sam Mendes},Daniel Craig
3,49026,"{David Monahan, Shane Nolan, Justin Michael Wo...",{Christopher Nolan},Christian Bale
4,49529,"{Nicholas Woodeson, Dominic West, Joseph Billi...",{Andrew Stanton},Taylor Kitsch


## Build actors index DataFrame
Get actors index and most relevant features

In [10]:
actor_cols = ["name", "gender", "order"]
actors_idx_df = json_normalize(actors_series)[actor_cols]
actors_idx_df = (
    actors_idx_df.rename(columns={actor_cols[0]: actors_col})
    .groupby(actors_col)
    .aggregate({actor_cols[1]: max, actor_cols[2]: list})
    .reset_index()
    .reset_index()
    .rename(columns={"index": "actor_id"})
)
actors_idx_df.head()

,actor_id,actors,gender,order
0,0,"""Weird Al"" Yankovic",0,[0]
1,1,50 Cent,2,[0]
2,2,AJ Michalka,1,[0]
3,3,Aamir Khan,2,"[0, 0]"
4,4,Aaron Abrams,2,[0]


## Build actors_index-credits DataFrame
Add actor features to the credits DataFrame

In [11]:
actors_idx_credits_df = pr_credits_df.merge(
    actors_idx_df, right_on=actors_col, left_on=actors_col, how="inner"
)
actors_idx_credits_df.head(20)

,movie_id,cast,crew,actors,actor_id,gender,order
0,19995,"{Tamica Washington-Miller, Hanniyah Muhammad, ...",{James Cameron},Sam Worthington,1653,2,"[0, 0, 0, 0]"
1,57165,"{Toby Kebbell, Kathryn Carpenter, Bill Nighy, ...",{Jonathan Liebesman},Sam Worthington,1653,2,"[0, 0, 0, 0]"
2,18823,"{Vincent Regan, David Kennedy, Jason Flemyng, ...",{Louis Leterrier},Sam Worthington,1653,2,"[0, 0, 0, 0]"
3,49527,"{John Dossett, Don Castro, Afton Williamson, E...",{Asger Leth},Sam Worthington,1653,2,"[0, 0, 0, 0]"
4,285,"{Takayo Fischer, Ho-Kwan Tse, Mackenzie Crook,...",{Gore Verbinski},Johnny Depp,926,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,58,"{Ho-Kwan Tse, Mackenzie Crook, Peter Donald Ba...",{Gore Verbinski},Johnny Depp,926,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,57201,"{Mason Cook, Harry Treadaway, Matthew Page, Ra...",{Gore Verbinski},Johnny Depp,926,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,1865,"{Deobia Oparei, Keith Richards, Stephen Graham...",{Rob Marshall},Johnny Depp,926,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,241259,"{Geraldine James, Lauren Beacham, Matt Wilman,...",{James Bobin},Johnny Depp,926,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,118,"{Geoffrey Holder, David Morris, Deep Roy, Mark...",{Tim Burton},Johnny Depp,926,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Build movies index DataFrame
Re-index movies 

In [12]:
movie_id_col = "movie_id"
title_col = "title"
new_movie_id_col = "new_movie_id"
movies_idx_df = (
    pr_movies_df[[movie_id_col, title_col]]
    .drop_duplicates()
    .reset_index()
    .rename(columns={"index": new_movie_id_col})
)
movies_idx_df.head()

,new_movie_id,movie_id,title
0,0,19995,Avatar
1,1,285,Pirates of the Caribbean: At World's End
2,2,206647,Spectre
3,3,49026,The Dark Knight Rises
4,4,49529,John Carter


In [13]:
movies_idx_csv_df = movies_idx_df.set_index(new_movie_id_col)
movies_idx_csv_df.to_csv("movies_idx.csv")

## Build actors-movies-credits DataFrame
Get actors from each movie and append their respective credits

In [14]:
actors_movies_credits_df = actors_idx_credits_df.merge(
    pr_movies_df, right_on=movie_id_col, left_on=movie_id_col, how="inner"
)
actors_movies_credits_df = (
    actors_movies_credits_df.merge(
        movies_idx_df.drop(columns=title_col),
        right_on=movie_id_col,
        left_on=movie_id_col,
        how="inner",
    )
    .drop(columns=movie_id_col)
    .rename(columns={new_movie_id_col: movie_id_col})
)
actors_movies_credits_df.head()

,cast,crew,actors,actor_id,gender,order,budget,genres,keywords,original_language,...,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,movie_id
0,"{Tamica Washington-Miller, Hanniyah Muhammad, ...",{James Cameron},Sam Worthington,1653,2,"[0, 0, 0, 0]",237000000,"{Fantasy, Action, Adventure, Science Fiction}","{3d, alien, cgi, space war, mind and soul, fut...",en,...,"{United Kingdom, United States of America}",2009-12-10,2787965087,162.0,"{English, Español}",Released,Avatar,7.2,11800,0
1,"{Toby Kebbell, Kathryn Carpenter, Bill Nighy, ...",{Jonathan Liebesman},Sam Worthington,1653,2,"[0, 0, 0, 0]",150000000,{Adventure},"{3d, ares, mythology, greek mythology, ancient...",en,...,"{United States of America, Spain}",2012-03-27,301000000,99.0,{English},Released,Wrath of the Titans,5.5,1431,132
2,"{Vincent Regan, David Kennedy, Jason Flemyng, ...",{Louis Leterrier},Sam Worthington,1653,2,"[0, 0, 0, 0]",125000000,"{Fantasy, Action, Adventure}","{3d, mythological beast, sea monster, medusa, ...",en,...,{United States of America},2010-04-01,232713139,106.0,{English},Released,Clash of the Titans,5.6,2233,206
3,"{John Dossett, Don Castro, Afton Williamson, E...",{Asger Leth},Sam Worthington,1653,2,"[0, 0, 0, 0]",42000000,"{Action, Thriller, Crime}","{escaped prisoner, diamond heist, deception, r...",en,...,{United States of America},2012-01-13,46221189,102.0,{English},Released,Man on a Ledge,6.2,925,1146
4,"{Takayo Fischer, Ho-Kwan Tse, Mackenzie Crook,...",{Gore Verbinski},Johnny Depp,926,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",300000000,"{Fantasy, Action, Adventure}","{love of one's life, shipwreck, traitor, ship,...",en,...,{United States of America},2007-05-19,961000000,169.0,{English},Released,Pirates of the Caribbean: At World's End,6.9,4500,1


## Build actors aggregated DataFrame

In [15]:
get_union = lambda s: reduce(set.union, s)
actor_id_col = "actor_id"
actors_agg_df = (
    actors_movies_credits_df.groupby(actors_col, as_index=False)
    .aggregate(
        {
            "movie_id": set,
            "cast": get_union,
            "crew": get_union,
            "actor_id": max,
            "gender": max,
            "budget": "mean",
            "genres": get_union,
            "keywords": get_union,
            "original_language": set,
            "popularity": "mean",
            "production_companies": get_union,
            "production_countries": get_union,
            "release_date": list,
            "revenue": "mean",
            "runtime": sum,
            "spoken_languages": get_union,
            "status": list,
            "title": set,
            "vote_average": "mean",
            "vote_count": "mean",
        }
    )
    .set_index(actor_id_col)
)
actors_agg_df.head()

,actors,movie_id,cast,crew,gender,budget,genres,keywords,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
actor_id,,,,,,,,,,,,,,,,,,,,
0,"""Weird Al"" Yankovic",{3619},"{Emo Philips, Trinidad Silva, David Bowe, Vict...",{Jay Levey},0,5000000.0,{Comedy},"{loser, satire, music spoof, tv station, india...",{en},4.070044,"{Orion Pictures, Cinecorp}",{United States of America},[1989-07-21],0.0,97.0,{English},[Released],{UHF},6.7,89.0
1,50 Cent,{1233},"{Adewale Akinnuoye-Agbaje, Tory Kittles, Lawre...",{Jim Sheridan},2,40000000.0,{Drama},"{attempted murder, musical, drug, career, rapp...",{en},12.885985,"{MTV Films, Paramount Pictures, Cent Productio...",{United States of America},[2005-11-09],46442528.0,117.0,{English},[Released],{Get Rich or Die Tryin'},6.3,186.0
2,AJ Michalka,{4203},"{Emma Catherwood, James Denton, Kevin Pollak, ...",{Brad J. Silverman},1,0.0,"{Music, Drama}",{christian},{en},2.160586,"{Coram Deo Studios, Birchwood Pictures}",{United States of America},[2013-10-04],0.0,102.0,{English},[Released],{Grace Unplugged},6.0,24.0
3,Aamir Khan,"{3543, 1023}","{Atul Kulkarni, Siddharth, Kirron Kher, Nandit...","{Rakeysh Omprakash Mehra, Deepa Mehta}",2,1100000.0,{Drama},"{celebration, students' movement, woman direct...",{hi},3.226925,"{Flicks Motion Pictures, UTV Motion Pictures, ...","{India, Canada}","[1998-09-16, 2006-01-26]",6015561.5,258.0,{हिन्दी},"[Released, Released]","{Earth, Rang De Basanti}",6.9,53.0
4,Aaron Abrams,{2242},"{Tatiana Maslany, Greg Kinnear, Lauren Graham,...",{Marc Abraham},2,0.0,{Drama},{biography},{en},3.350297,"{Universal Pictures, Strike Entertainment, Spy...",{United States of America},[2008-09-07],0.0,119.0,{English},[Released],{Flash of Genius},6.3,59.0


In [16]:
actors_agg_df.to_pickle("actors_agg_df.pkl")

## Tests

In [17]:
# Cast should be set
actors_agg_df = pd.read_pickle("actors_agg_df.pkl")
type(actors_agg_df.loc[0, "cast"])

set